In [1]:
import pandas as pd
import numpy as np
from numpy import hstack
import warnings
from datetime import datetime
import import_ipynb
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import talib as ta

from redis_cli import RedisClient

from tqdm import tqdm
tqdm.pandas()
warnings.simplefilter('ignore')

from file_loader import get_data, store_to_file
from features import FeatureEngineering
import utils as ut
from data_loader import load_forex_data, load_forex_data2

2025-01-06 13:01:06.763059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736150466.797276   10624 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736150466.807423   10624 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-06 13:01:06.839997: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


yfinance version: 0.2.50


In [2]:
# r = RedisClient(db=1, username='usr_redis', password='usr_pwd')
# r.test_connection()

In [3]:
period=-(datetime.now() - datetime(2016, 1, 1)).days
time_interval='1d'
tickers = ['EURUSD=X', 'GBPUSD=X', 'USDCHF=X', 'USDJPY=X']
forex_tickers = ['EURUSD', 'GBPUSD', 'USDCHF', 'USDJPY']

In [4]:
data_dir = load_forex_data(tickers, period, time_interval)

Start load forex data, tickers ['EURUSD=X', 'GBPUSD=X', 'USDCHF=X', 'USDJPY=X'], interval: 1d, from: 2016-01-01 13:01:09.727893


[*********************100%***********************]  4 of 4 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2348 entries, 2016-01-01 to 2025-01-06
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   (GBPUSD=X, Open)       2348 non-null   float64
 1   (GBPUSD=X, High)       2348 non-null   float64
 2   (GBPUSD=X, Low)        2348 non-null   float64
 3   (GBPUSD=X, Close)      2348 non-null   float64
 4   (GBPUSD=X, Adj Close)  2348 non-null   float64
 5   (GBPUSD=X, Volume)     2348 non-null   int64  
 6   (USDCHF=X, Open)       2348 non-null   float64
 7   (USDCHF=X, High)       2348 non-null   float64
 8   (USDCHF=X, Low)        2348 non-null   float64
 9   (USDCHF=X, Close)      2348 non-null   float64
 10  (USDCHF=X, Adj Close)  2348 non-null   float64
 11  (USDCHF=X, Volume)     2348 non-null   int64  
 12  (USDJPY=X, Open)       2348 non-null   float64
 13  (USDJPY=X, High)       2348 non-null   float64
 14  (USDJPY=X, Low)        2348 non-null  

In [5]:
def compare_and_plot(data, column_name1, column_name2):
    df = data.copy()
    X_train = df[column_name1]
    y_train = df[column_name2]

    # without normalization
    model_no_norm = ut.linear_regression(X_train, y_train)
    weights_no_norm = model_no_norm.coef_

    # normalize data
    prices_norm, volume_norm = ut.normalize_MinMax_by_column(X_train, y_train)
    # print(prices_norm)

    model_with_norm = ut.linear_regression(prices_norm, volume_norm)
    weights_with_norm = model_with_norm.coef_

    # Plotting the weights
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    axes[0].bar(['Price', 'Volume'], weights_no_norm, color='red')
    axes[0].set_title('Weigth without normalization')
    axes[0].set_ylabel('Model weghts')

    axes[1].bar(['Price', 'Volume'], weights_with_norm, color='green')
    axes[1].set_title('Weigth after normalization')
    axes[1].set_ylabel('Model weghts')

    plt.tight_layout()
    plt.show()
    

In [6]:
# store_type = file, redis
def merge_and_store_data(new_df, key, store_type='file', compress=False):
    if store_type == 'redis':
        saved_data = r.get_key(key)
    else:
        saved_data = get_data('_data_store', key, compress=compress)

    merged_df = ut.validate_duplicate_and_merge(saved_data, new_df)

    if store_type == 'redis':
        r.set_key(key, merged_df)
    else:
        store_to_file(merged_df, key, compress=compress)

    merged_df.dropna(inplace=True)
    return merged_df

In [7]:
def plot_cv_indices(cv, n_splits, X, y, date_col = None):
    """Create a sample plot for indices of a cross-validation object."""
    
    fig, ax = plt.subplots(1, 1, figsize = (11, 7))
    
    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=10, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)


    # Formatting
    yticklabels = list(range(n_splits))
    
    if date_col is not None:
        tick_locations  = ax.get_xticks()
        tick_dates = [" "] + date_col.iloc[list(tick_locations[1:-1])].astype(str).tolist() + [" "]

        tick_locations_str = [str(int(i)) for i in tick_locations]
        new_labels = ['\n\n'.join(x) for x in zip(list(tick_locations_str), tick_dates) ]
        ax.set_xticks(tick_locations)
        ax.set_xticklabels(new_labels)
    
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+0.2, -.2])
    ax.legend([Patch(color=cmap_cv(.8)), Patch(color=cmap_cv(.02))],
              ['Testing set', 'Training set'], loc=(1.02, .8))
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)

### Processing

In [8]:
def show_importance(model, model_func, params):
    importance_function = model.coef_[0]
    if model_func in [ut.ModelFunc.XGBOOST_CLASS, ut.ModelFunc.DECISION_TREE_CLASS, ut.ModelFunc.RANDOM_FOREST_CLASS, \
                      ut.ModelFunc.KNN_CLASS, ]:
        importance_function = model.feature_importances_
    if model_func in [ut.ModelFunc.CATBOOST_CLASS, ]:
        importance_function = model.get_feature_importance()

    ut.top_n_weighted_factors(importance_function, params['features'], params['top'])
    return importance_function

In [9]:
### Blending, Stacking, Ensemble
def fit_models(model_funcs, X_train, y_train, X_val=None, y_val=None):
    models = list()
    for model_func in model_funcs:
        params = ut.get_model_params(model_func)
        if X_val is None or y_val is None:
            model = ut.model_fit(model_func, X_train, y_train, params)
        else:
            if model_func is ut.ModelFunc.CATBOOST_CLASS:
                params = dict(params, early_stopping_rounds=50)
                model = ut.model_fit_with_eval(model_func, X_train, y_train, eval_set=(X_val, y_val), params=params)
            else:
                model = ut.model_fit(model_func, X_train, y_train, params)
        models.append(model)

    return models

def predict_models(models, X_train, X_val, X_test):
    models_proba = list()
    for model in models:
        models_proba.append({
            'train': np.array(model.predict_proba(X_train)[:, 1]),
            'val': np.array(model.predict_proba(X_val)[:, 1]),
            'test': np.array(model.predict_proba(X_test)[:, 1])
        })

    return models_proba


def blending_pred(*args):
    return sum(args) / len(args)
    
def stacking_pred(*args):
    return np.column_stack(args)

# stacking_pred(1,2,3)

In [15]:
params = {
    'emaf': 20,
    'emam': 100,
    'emas': 150,
    'rsi': 14,
    'macd': [12, 26, 9],
    'max_train_size': 90,
    'test_size': 30
}

trend_indicators = [ 'emaf', 'emam', 'emas', 'rsi', 'macd', 'adx']
lag_periods = 7 #3 # depends on timeframe, 7 days - ???
min_outliers=.23
max_outliers=.77
threshold = 0.6 # ???
use_stacking = True
use_blending = False

fe = FeatureEngineering(params)

In [11]:
def predict_ensemble(model_funcs, data_with_features, features):
    for train_data, val_data, test_data in tqdm(fe.split_data(data_with_features)):
        X_train, y_train = train_data[features], train_data['Target']
        X_val, y_val     = val_data[features], val_data['Target']
        X_test, y_test   = test_data[features], test_data['Target']

        ## Data normalization
        # X_train_scaled, X_val_scaled, X_test_scaled = ut.normalize_MinMaxScaler(X_train, X_val, X_test)
        X_train_scaled, X_val_scaled, X_test_scaled = ut.normalize_StandardScaler(X_train, X_val, X_test)
    
        ## Modeling
        # models = models_fit(model_funcs, X_train_scaled, y_train, X_val=X_val, y_val=y_val)
        models = fit_models(model_funcs, X_train_scaled, y_train)

        ## Prediction on train, val and test samples
        predict_dict = predict_models(models, X_train_scaled, X_val_scaled, X_test_scaled)

        yield ( predict_dict, y_train, y_val, y_test )


In [16]:
def get_models():
    models = list()
    # models.append(ut.ModelFunc.LOGISTIC_REG)
    # models.append(ut.ModelFunc.LINEAR_REG)
    # models.append(ut.ModelFunc.KNN_REG)
    # models.append(ut.ModelFunc.DECISION_TREE_REG)
    # models.append(ut.ModelFunc.RANDOM_FOREST_REG)
    # models.append(ut.ModelFunc.CATBOOST_REG)
    # models.append(ut.ModelFunc.XGBOOST_REG)

    # models.append(ut.ModelFunc.XGBOOST_CLASS)
    # models.append(ut.ModelFunc.CATBOOST_CLASS)

    # models.append(ut.ModelFunc.RANDOM_FOREST_CLASS)
    models.append(ut.ModelFunc.DECISION_TREE_CLASS)
    models.append(ut.ModelFunc.KNN_CLASS)
    return models

In [17]:
symbols =['EURUSD']
for symbol in symbols:
# for name in tqdm(tickers):
    print(f'=== symbol: {symbol}, stacking: {use_stacking}, blending: {use_blending} ===')

    data = get_data(data_dir, symbol)
    data.drop(columns=['chg', 'vol_chg'], inplace=True) # Could it be as features ?

    df = fe.clear_invalid_targets(fe.add_target(fe.enrich_with_indicators(data), lag_periods))
    # df = fe.clear_invalid_targets(fe.add_target2(fe.enrich_with_indicators(data)))
    df = fe.validate_outliers(df, 'Close', min_outliers, max_outliers)
    # # print(df.isnull().sum())
    
    ## Store data
    # df = merge_and_store_data(df, symbol, compress=True) # Store data
    # print(df.isnull().sum())

    ## Add features
    OHLCV = ['Open', 'High', 'Low', 'Close']

    # Trend features
    data_with_trend, new_trend_features = fe.create_trend_features(df, OHLCV, lag_periods)
    # print(data_with_trend.isnull().sum())
    features = new_trend_features + trend_indicators  + ['Target']
    data_with_features = data_with_trend[features + ['Date']]


    # Rolling features
    # window_sizes = [7, 14, 30]
    # data_with_rolling, new_rolling_features = fe.create_rolling_features(df, OHLCV, window_sizes)
    # features = new_rolling_features + trend_indicators + ['Target']
    # data_with_features = data_with_rolling[features + ['Date']]

    # print(len(data_with_features))
    # print(data_with_features.isnull().sum())
    data_with_features.set_index('Date', inplace=True)
    # display(data_with_features.tail(10))


    model_funcs = get_models()
    print(f'Models: {model_funcs}')

    ## Split, predict
    train_pred_prob = list()
    val_pred_prob = list()
    test_pred_prob = list()
    y_train_total = pd.DataFrame()
    y_val_total = pd.DataFrame()
    y_test_total = pd.DataFrame()

    for predict_dict, y_train, y_val, y_test in predict_ensemble(model_funcs, data_with_features, features):
        y_train_total = pd.concat([y_train_total, y_train])
        y_val_total = pd.concat([y_val_total, y_val])
        y_test_total = pd.concat([y_test_total, y_test])

        train_pred_prob.append([d['train'] for d in predict_dict][0])
        val_pred_prob.append([d['val'] for d in predict_dict][0])
        test_pred_prob.append([d['test'] for d in predict_dict][0])

    ## 2D-array
    train_pred_prob = hstack(train_pred_prob)
    val_pred_prob = hstack(val_pred_prob)
    test_pred_prob = hstack(test_pred_prob)


    ## Final model using whole data
    if use_stacking:
        stacked_train_X = stacking_pred(train_pred_prob).reshape(-1,1)
        stacked_val_X = stacking_pred(val_pred_prob).reshape(-1,1)
        stacked_test_X = stacking_pred(test_pred_prob).reshape(-1,1)
        
        final_model_func = ut.ModelFunc.LOGISTIC_REG 
        final_model = fit_models([final_model_func], stacked_train_X, y_train_total)[0]
        predict_dict = predict_models([final_model], stacked_train_X, stacked_val_X, stacked_test_X)

    elif use_blending:
        blended_train_X = stacking_pred(train_pred_prob).reshape(-1,1)
        blended_val_X = stacking_pred(val_pred_prob).reshape(-1,1)
        blended_test_X = stacking_pred(test_pred_prob).reshape(-1,1)

        final_model_func = ut.ModelFunc.LOGISTIC_REG
        final_model = fit_models([final_model_func], blended_train_X, y_train_total)[0]
        predict_dict = predict_models([final_model], blended_train_X, blended_val_X, blended_test_X)

    else:
        final_model_func = model_funcs[0]
        final_model = fit_models([final_model_func], train_pred_prob.reshape(-1,1), y_train_total)[0]
        predict_dict = predict_models([final_model], train_pred_prob.reshape(-1,1), val_pred_prob.reshape(-1,1),\
                                       test_pred_prob.reshape(-1,1))

        # params = {
        #     'features': features,
        #     'top': 5,
        # }
        # importance_features = show_importance(final_model, final_model_func, params)

    ensemble_train = [d['train'] for d in predict_dict][0]
    ensemble_val = [d['val'] for d in predict_dict][0]
    ensemble_test = [d['test'] for d in predict_dict][0]

    # ## Display metrics, ROC AUC for train, val and test samples
    train_roc_auc = ut.roc_auc_score_metric(y_train_total, ensemble_train)
    val_roc_auc = ut.roc_auc_score_metric(y_val_total, ensemble_val)
    test_roc_auc = ut.roc_auc_score_metric(y_test_total, ensemble_test)
  
    print('=== Train sample metrics ===')
    print(f'ROC AUC: {train_roc_auc:.4f}')
    print(ut.calculate_metrics_table(y_train_total, ensemble_train))

    print('=== Val sample metrics ===')
    print(f'ROC AUC: {val_roc_auc:.4f}')
    print(ut.calculate_metrics_table(y_val_total, ensemble_val))

    print('=== Test sample metrics ===')
    print(f'ROC AUC: {test_roc_auc:.4f}')
    print(ut.calculate_metrics_table(y_test_total, ensemble_test))
    print('===========================')


=== symbol: EURUSD, stacking: True, blending: False ===
Outliers detected: 0
            Date      Open      High       Low     Close  Volume       rsi  \
2331  2024-12-12  1.050685  1.052997  1.046485  1.050685       0  0.796998   
2332  2024-12-13  1.047406  1.052299  1.045435  1.047406       0  0.740678   
2333  2024-12-16  1.051005  1.052410  1.047504  1.051005       0  0.839565   
2334  2024-12-17  1.051735  1.053286  1.047998  1.051735       0  0.859156   
2335  2024-12-18  1.049505  1.051525  1.046781  1.049505       0  0.814779   
2336  2024-12-19  1.035025  1.042220  1.035250  1.035025       0  0.598561   
2337  2024-12-20  1.036495  1.043308  1.034426  1.036495       0  0.638727   
2338  2024-12-23  1.043308  1.044823  1.038594  1.043308       0  0.809543   
2339  2024-12-24  1.040583  1.041124  1.038745  1.040583       0  0.768695   
2340  2024-12-25  1.040258  1.043297  1.040150  1.040258       0  0.763750   

          emaf      emam      emas  ...  Low_ratio_1  Low_log_di

69it [00:01, 36.56it/s]


=== Train sample metrics ===
ROC AUC: 1.0000
   Cutoff  Precision  Recall  Accuracy  F1-Score
0    50.0      100.0   100.0     100.0     100.0
1    60.0      100.0   100.0     100.0     100.0
2    70.0      100.0   100.0     100.0     100.0
3    80.0      100.0   100.0     100.0     100.0
=== Val sample metrics ===
ROC AUC: 1.0000
   Cutoff  Precision  Recall  Accuracy  F1-Score
0    50.0      100.0   100.0     100.0     100.0
1    60.0      100.0   100.0     100.0     100.0
2    70.0      100.0   100.0     100.0     100.0
3    80.0      100.0   100.0     100.0     100.0
=== Test sample metrics ===
ROC AUC: 1.0000
   Cutoff  Precision  Recall  Accuracy  F1-Score
0    50.0      100.0   100.0     100.0     100.0
1    60.0      100.0   100.0     100.0     100.0
2    70.0      100.0   100.0     100.0     100.0
3    80.0      100.0   100.0     100.0     100.0
